In [44]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import warnings
warnings.filterwarnings('ignore')

### Haven`t find better way to download encoder

In [3]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k',
    split=(tfds.Split.TRAIN, tfds.Split.TEST),
    with_info=True, as_supervised=True
)

In [4]:
encoder = info.features['text'].encoder

In [17]:
[f for f in os.listdir('DATA') if not f.startswith('.')]

['test_submission.csv',
 'embeddings',
 'test.csv',
 'train.csv',
 'sample_submission.csv']

In [50]:
def fetch_data_train(path='DATA'):
    data = pd.read_csv(os.path.join(path, 'train.csv'))
    
    X = data.drop(['qid', 'target'], axis=1).values
    y = data['target'].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    return X_train, X_test, y_train, y_test

def fetch_data_test(path='DATA'):
    train = pd.read_csv(os.path.join(path, 'train.csv'))
    test = pd.read_csv(os.path.join(path, 'test.csv'))
    
    X_train = train.drop(['qid', 'target'], axis=1).values
    y_train = train['target'].values
    X_test = train.drop(['qid'], axis=1).values
    
    return X_train, X_test, y_train

In [51]:
X_train, X_test, y_train, y_test = fetch_data_train()

In [52]:
X_train

array([['What is the best possible way to utilise 15 minutes of reading time during class 10 board exams?'],
       ['How can life fascinate a person?'],
       ['What is the duration of the longest female orgasm?'],
       ...,
       ['What influenced Warren Sapp to become an athlete?'],
       ['Regarding a convergent (destructive) tectonic plate, why does the forcing of an oceanic plate under a continental plate force magma up into a volcano?'],
       ['Will liberals ever grow up and quit acting like children towards President Trump?']],
      dtype=object)

In [42]:
example_sentence = X_train[0][0]
encoded_sentence = encoder.encode(example_sentence)
decoded_sentence = encoder.decode(encoded_sentence)

assert example_sentence == decoded_sentence

print('Original sentence:', decoded_sentence)
print('Encoded sentence', encoded_sentence)

Original sentence: What are the best ways to warm bread in the oven?
Encoded sentence [274, 29, 1, 175, 1766, 7, 1892, 7961, 2144, 2189, 11, 1, 1928, 413, 7992]


In [60]:
class EmbeddingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, encoder):
        self.encoder = encoder
        self.vocab_size = self.encoder.vocab_size
    
    def fit(self, X, y=None):
        pass
    
    def transform(self, X, y=None):
        X_transformed = list()
        for line in X.reshape(1, -1)[0]:
            X_transformed.append(
                self.encoder.encode(line)
            )
        return np.array(X_transformed).reshape(-1,1)

In [63]:
pipeline = Pipeline([
    ('embedding', EmbeddingTransformer(encoder=encoder)),
])